In [1]:
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
findspark.init()

In [3]:
spark = SparkSession.builder.appName('Averages').getOrCreate()

24/07/03 10:21:15 WARN Utils: Your hostname, Bisheshs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.13.163.225 instead (on interface en0)
24/07/03 10:21:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/03 10:21:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/07/03 10:21:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
df = spark.read.csv('../datasets/fc_transaction_base.csv',header='true', inferSchema='true')

24/07/03 10:21:28 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [6]:
df.printSchema()
df.show(5)

root
 |-- tran_date: string (nullable = true)
 |-- account_number: string (nullable = true)
 |-- branch: integer (nullable = true)
 |-- product: string (nullable = true)
 |-- lcy_amount: double (nullable = true)
 |-- transaction_code: string (nullable = true)
 |-- description1: string (nullable = true)
 |-- dc_indicator: string (nullable = true)
 |-- is_salary: integer (nullable = true)

+----------+-------------------+------+-------+----------+----------------+--------------------+------------+---------+
| tran_date|     account_number|branch|product|lcy_amount|transaction_code|        description1|dc_indicator|is_salary|
+----------+-------------------+------+-------+----------+----------------+--------------------+------------+---------+
|09/06/2020|02XYZXYZ10017529992|    15|    SBA|   14500.0|              CI|ATM WDL /00463701...|    withdraw|        0|
|14/06/2020|02XYZXYZ10017529992|    15|    SBA|   20000.0|              CI|ATM WDL /00463701...|    withdraw|        0|
|01/06/20

In [7]:
df = df.withColumn("tran_date", to_date(col("tran_date"), "yyyy-MM-dd"))

In [8]:
df = df.withColumn("month", month(col("tran_date")))

In [9]:
df1 = df.fillna(0)

In [10]:
stats_df = df1.groupBy("account_number", "month").agg(
    avg("lcy_amount").alias("avg_lcy_amount"),
    stddev("lcy_amount").alias("stddev_lcy_amount"),
    variance("lcy_amount").alias("variance_lcy_amount")
)
stats_df=stats_df.fillna(0)

In [11]:
stats_df.show(5)

+-------------------+-----+------------------+------------------+--------------------+
|     account_number|month|    avg_lcy_amount| stddev_lcy_amount| variance_lcy_amount|
+-------------------+-----+------------------+------------------+--------------------+
|02XYZXYZ10017537924|    0| 4424.599428571428| 9198.405535223752| 8.461066439043497E7|
|02XYZXYZ10017528599|    0|22662.531333333336|11968.867662633158|1.4325379312562573E8|
|02XYZXYZ10017510834|    0| 39827.44666666666|  90745.7126763002| 8.234784369129632E9|
|02XYZXYZ10017537222|    0| 14142.78172413793|11614.945410882749| 1.349069568977862E8|
|02XYZXYZ10017558914|    0| 5876.409090909091| 5650.534723759255|3.1928542664409082E7|
+-------------------+-----+------------------+------------------+--------------------+
only showing top 5 rows



In [12]:
stats1_df = stats_df.groupBy("account_number").agg(
    avg("avg_lcy_amount").alias("o_avg_lcy_amount"),
    avg("stddev_lcy_amount").alias("o_stddev_lcy_amount"),
    avg("variance_lcy_amount").alias("o_variance_lcy_amount")
)
stats1_df=stats1_df.fillna(0)

In [13]:
stats1_df.show(10)

+-------------------+------------------+-------------------+---------------------+
|     account_number|  o_avg_lcy_amount|o_stddev_lcy_amount|o_variance_lcy_amount|
+-------------------+------------------+-------------------+---------------------+
|02XYZXYZ10017507450| 11457.23129032258| 13630.693254894448|   1.85795798609025E8|
|02XYZXYZ10017287701|19466.004035087724|  55770.44472948584|  3.110342505324635E9|
|02XYZXYZ10017534270|11464.473999999998|  9792.489620721588|  9.589285297194003E7|
|02XYZXYZ10017518608| 5441.291666666667|   8426.50711934378|   7.10060222323514E7|
|02XYZXYZ10017529992|11816.281851851854|  14810.80423224391| 2.1935992200585413E8|
|02XYZXYZ10017544491|5108.7814285714285|  7026.320826725816|  4.936918436008096E7|
|02XYZXYZ10017272701| 75566.42873684214| 384177.99371728394| 1.475927308566374...|
|02XYZXYZ10017520808|     32186.0278125|   93144.5591676245|  8.675908902531101E9|
|02XYZXYZ10017504738|22481.000000000004|  80203.99289133798|  6.432680475713793E9|
|02X

In [16]:
df = spark.read.csv('../datasets/fc_account_master.csv',header='true', inferSchema='true')

In [17]:
df.show(5)

+-------------------+-------------+-------+----------------+-------------+---------------+-----------+
|     account_number|customer_code|product|product_category|acc_open_date|acc_closed_date|active_flag|
+-------------------+-------------+-------+----------------+-------------+---------------+-----------+
|02XYZXYZ10015592101|    KL0255921|    SBA|           SBPPS|   1999-03-10|           NULL|          0|
|02XYZXYZ10015593701|    KL0255937|    SBA|           SBPPS|   1999-03-10|           NULL|          0|
|02XYZXYZ10015593801|    KL0255938|    SBA|           SBPPS|   1999-03-10|           NULL|          0|
|02XYZXYZ10015594801|    KL0255948|    SBA|           SBANU|   1999-03-22|           NULL|          0|
|02XYZXYZ10015597601|    KL0255976|    SBA|           SBPPS|   1999-04-22|           NULL|          0|
+-------------------+-------------+-------+----------------+-------------+---------------+-----------+
only showing top 5 rows



In [18]:
stats2_df = stats1_df.join(df, on='account_number', how='inner')

In [19]:
stats3_df = stats2_df.select('account_number', 'customer_code','o_avg_lcy_amount','o_stddev_lcy_amount','o_variance_lcy_amount')

In [20]:
stats3_df=stats3_df.show(10)

+-------------------+-------------+------------------+-------------------+---------------------+
|     account_number|customer_code|  o_avg_lcy_amount|o_stddev_lcy_amount|o_variance_lcy_amount|
+-------------------+-------------+------------------+-------------------+---------------------+
|02XYZXYZ10017507450|    KL2027115| 11457.23129032258| 13630.693254894448|   1.85795798609025E8|
|02XYZXYZ10017287701|    PB2075365|19466.004035087724|  55770.44472948584|  3.110342505324635E9|
|02XYZXYZ10017534270|    PB2097839|11464.473999999998|  9792.489620721588|  9.589285297194003E7|
|02XYZXYZ10017518608|    KL2054409| 5441.291666666667|   8426.50711934378|   7.10060222323514E7|
|02XYZXYZ10017529992|    PB2083592|11816.281851851854|  14810.80423224391| 2.1935992200585413E8|
|02XYZXYZ10017544491|    PB2143036|5108.7814285714285|  7026.320826725816|  4.936918436008096E7|
|02XYZXYZ10017272701|    KL2009222| 75566.42873684214| 384177.99371728394| 1.475927308566374...|
|02XYZXYZ10017520808|    PB206